# Cut surface of paracetamol crystal

## By PyMatGen
The following example using PyMatGen does not work - Even though PyMatGen allows maintaining bonds, the cutting surface is still assumed to be, in general, a flat surface, which cannot be used to the case of Form I paracetamol since during the cutting the molecule is divided into fragments, leaving some bonds completely on the other side of the plane

In [1]:
from pymatgen.io import cif
from pymatgen.core.surface import Slab, SlabGenerator, generate_all_slabs, Structure, Lattice, ReconstructionGenerator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure, IStructure
from pymatgen.analysis.adsorption import *


f1bulk = cif.CifParser('F1-bulk.cif').get_structures(primitive=False)[0]
f1bulk = SpacegroupAnalyzer(f1bulk).get_conventional_standard_structure()
f1bulk.make_supercell([[1, 0, 0], 
                       [0, 4, 0], 
                       [0, 0, 1]])
bulk = f1bulk
interface = [0, 1, 0]
slab_thickness = 24.0
vac_thickness = 10.0
bond_list = {
    ('C', 'C'): 1.60,
    ('C', 'O'): 1.40,
    ('C', 'H'): 1.10,
    ('C', 'N'): 1.45,
    ('H', 'O'): 1.05,
    ('H', 'N'): 1.10
}

slab010 = SlabGenerator(bulk, 
                        miller_index=interface, 
                        min_slab_size=slab_thickness,
                        min_vacuum_size=vac_thickness, 
                        center_slab=True)
slab_struc = slab010.get_slabs(bonds = bond_list)
print(slab_struc)

[]


## PyMatGen modified

A brief idea: 

1. Identify the topology of molecules and reorder the atom list to get a list of molecule objects. The sequence of atoms should be consistent among all the molecules. 

2. Grab a representive point of the molecule, such as an atom or the mass center, then get the reduced cell with the same representive points only.

3. Cut surfaces by PyMatGen

4. According to the indeices, restore complete molecules back to get the complete cell. 

For Step 1, the 'ATOMORDE' keyword of CRYSTAL can be used (and is used here). Alternatively, it seems to be possible to do it within PyMatGen, but not directly. Refer to [pymatgen.core.bonds.CovalentBond.is_bonded](https://pymatgen.org/pymatgen.core.bonds.html) and [pymatgen.core.sites.PeriodicSite.is_periodic_image](https://pymatgen.org/pymatgen.core.sites.html). 

For Step 2 and 3, it is hard to find a really representative point. The problem with mass center is that it hardly occupies the meaningful Wyckoff position, changing the symmetry of the reduced cell. If the same atom of different molecules is used, problems still exist: Consider a case that 2 molecules are parallel but aligne in opposite directions - that makes PyMatGen regards them to be 2 separate surfaces. Run the code for more information. 

**Note** PyMatGen version: 2023.1.30. Earlier versions might lead to errors with Slabgenerator object. 

In [1]:
from CRYSTALpytools.crystal_io import Crystal_output

class Output_reader(Crystal_output):
    def __init__(self, file):
        super(Output_reader, self).read_cry_output(file)
    
    def read_molecule_in_crystal(self):
        """
        Read the reordered list of atomic coordinates (Keyword ATOMORDE). 
        Return to a list of PyMatGen molecule objects
        """
        import numpy as np
        import re
        import sys
        import warnings
        from pymatgen.core.structure import Molecule
        
        if not hasattr(self, 'data'):
            print('ERROR: Output file not specified.')
            sys.exit(1)
            
        molecule_list = []
        flag_bg_mole = False
        for idx_line, line in enumerate(self.data):
            if re.match(r'^\s+MOLECULAR FRAGMENT N\.\s+[0-9]+', line):
                flag_bg_mole = True
                species = []
                coords = []
            elif flag_bg_mole and re.match(r'^\s+[0-9]+', line):
                species.append(line.strip().split()[5])
                coords.append(line.strip().split()[6:])
            elif flag_bg_mole and len(line.strip().split()) == 0:
                flag_bg_mole = False
                species = np.array(species, dtype=int)
                coords = np.array(coords, dtype=float)
                molecule_list.append(Molecule(species, coords))
        
        if len(molecule_list) == 0:
            warnings.warn(
                "The input file: %s has no output by keyword 'ATOMORDE'. Check the input file"\
                % self.name, stacklevel=2
            )
        else:
            self.molecule_in_crystal = molecule_list
        
        return self

In [2]:
from pymatgen.core.surface import SlabGenerator
from pymatgen.core.structure import Structure, IStructure, IMolecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter
from CRYSTALpytools.convert import cry_out2pmg

output = Output_reader('form1-expt.out')
org_cell = cry_out2pmg(output)
output.read_molecule_in_crystal()
symm = SpacegroupAnalyzer(org_cell).get_space_group_number()

# Case 1: Use mass centre
mass_centers = [
    IMolecule(
        species=m.species, 
        coords=[[site.x, site.y, site.z] for site in m.sites]
    ).center_of_mass 
    for m in output.molecule_in_crystal
]

red_cell = Structure(
    lattice=org_cell.lattice,
    species=['X%i' % i for i in range(len(mass_centers))],
    coords=mass_centers,
    coords_are_cartesian=True
)

red_cell_conv = SpacegroupAnalyzer(red_cell).get_conventional_standard_structure()
CifWriter(red_cell_conv).write_file('massc-bulk.cif')

slabg = SlabGenerator(red_cell_conv, 
                      miller_index=[0, 1, 0], 
                      min_slab_size=2, 
                      min_vacuum_size=4, 
                      center_slab=True, 
                      in_unit_planes=True)
for ns, slab in enumerate(slabg.get_slabs()):
    CifWriter(slab).write_file('massc-%d.cif' % ns)

# Case 2: Use the first atom
red_cell = IStructure.from_spacegroup(
    sg=symm,
    lattice=org_cell.lattice,
    species=[str(output.molecule_in_crystal[0].species[0])],
    coords=[output.molecule_in_crystal[0].sites[0].coords],
    coords_are_cartesian=True
)

red_cell_conv = SpacegroupAnalyzer(red_cell).get_conventional_standard_structure()
CifWriter(red_cell_conv).write_file('atom1-bulk.cif')

slabg = SlabGenerator(red_cell_conv, 
                      miller_index=[0, 1, 0], 
                      min_slab_size=2, 
                      min_vacuum_size=4, 
                      center_slab=True, 
                      in_unit_planes=True)
for ns, slab in enumerate(slabg.get_slabs()):
    CifWriter(slab).write_file('atom1-%d.cif' % ns)

## A semi-automic solution

Paracetamol crystals are featured by hydrogen bond networks. The crystal is a vertical stack of hydrogen bond networks. Taking the Form I paracetamol as an example, the crystal is composed of 2 differently oriented hydrogen bond networks, which are stacked along the b axis. See the following figure:

![slab_struc-010](010-analysis.png)

A semi-automic solution is therefore proposed:

1. Analysis the periodic pattern and find the differently oriented surfaces. **The code does not recognize any rotation, mirror plane etc. to avoid drift in atomic coordinates**.  
2. In VESTA, from the bulk crystal, manually delete the atoms belonging to other networks and keep atoms of the desired network. The 2D periodicity should be kept.  
3. The 2D periodic hydrogen network is read in the following code blocks, as VASP POSCAR file written in Cartesian coordinates. The differently oriented hydrogen bond network is re-orginzed and arranged to generate the slab model. 

Note:

1. The relative distance between layer A and B cannot be established via this method. So independent A and B 2D periodic models should be generated from the same bulk model with the same origin. The distance between the same layer, i.e., A-A or B-B, equals the lattice parameter of the non-periodic dimension, which, in the case of Form I paracetamol, corresponds to *b*   
2. The sequence of layers is suggested to be 'bottom-up', i.e., lower layer first. The code cannot rearrange the sequence.
3. If necessary (and usually the case), rotate the lattice coordinates in VESTA to make the slab orientated to z-axis.  

In [1]:
def get_lattice(filename):
    """
    Read lattice vectors. Return to a 3*3 numpy array.
    """
    import numpy as np

    file = open(filename, 'r')
    data = file.readlines()
    file.close()
    shrink = float(data[1].strip().split()[0])
    lattice = [data[2].strip().split(), 
               data[3].strip().split(), 
               data[4].strip().split()]
    lattice = np.array(lattice, dtype=float) * shrink
    
    return lattice

def get_atom(filename):
    """
    Read atomic species and coordinates. Only Cartesian coordinates are allowed.
    Return to species (natom*1 list) and coords (natom*3 array)
    """
    import re
    import numpy as np
    
    file = open(filename, 'r')
    data = file.readlines()
    file.close()
    
    if not re.match(r'^\s*C', data[7]):
        print('ERROR: Only Cartesian coordinates are premitted.')
        return
    
    labels = data[5].strip().split()
    count_species = data[6].strip().split()
    count_species = np.array(count_species, dtype=int)
    
    bg_line = 8
    species = []
    coords = []
    for idx_label, label in enumerate(labels):
        for idx_a in range(bg_line, bg_line + count_species[idx_label]):
            species.append(label)
            coords.append(data[idx_a].strip().split())
        
        bg_line += count_species[idx_label]
    
    coords = np.array(coords, dtype=float)
    
    return species, coords


def get_slab(file_list, index, nlayer, same_layer_space, vacuum):
    """
    Expand lattice and combine atoms from different layers together to get the
    slab model.
    Note: The lattice used should be consistent. This is not checked by code.
    
    nlayer: Number of layers of the slab model
    index: Miller index of the slab surface
    same_layer_space: The length between same layers, in Angstrom
    vacuum: Thickness  of the vaccum layer, in Angstrom
    """
    from pymatgen.core.lattice import Lattice
    from pymatgen.core.structure import Structure
    import numpy as np
    
    species = []
    coords = []
    latt_mx = get_lattice(file_list[0])
    latt_pri = Lattice(latt_mx)
    for file in file_list:
        species.append(get_atom(file)[0])
        coords.append(get_atom(file)[1])
        
    # Get displacements in Cartesian coordinates
    space_unit = index * np.array(latt_pri.abc, dtype=float)
    space_unit = space_unit / np.linalg.norm(space_unit)
    space_xyz = np.dot(space_unit * same_layer_space / np.array(latt_pri.abc, dtype=float), latt_mx)
    # Displace atomic coordinates
    if nlayer > len(file_list):
        for idx_layer in range(len(file_list), nlayer):
            species.append(species[idx_layer % len(file_list)])
            space = space_xyz * (idx_layer // len(file_list))
            coords.append(coords[idx_layer % len(file_list)] + space)
        
        coords_tot = np.array(coords, dtype=float).reshape([-1, 3])
    else:
        coords_tot = np.array(coords[:nlayer], dtype=float).reshape([-1, 3])
        
    species_tot = []
    for i in range(nlayer):
        species_tot = species_tot + species[i]
    
    # Project the coordinates to elementary stacking vector, to get slab thickness
    thickness = vacuum + max(np.dot(coords_tot, space_unit)) - min(np.dot(coords_tot, space_unit))
    
    # Extend lattice matrix
    new_times_oldcell = space_unit * thickness / np.array(latt_pri.abc, dtype=float)
    for i in range(3):
        if abs(new_times_oldcell[i]) < 1e-4:
            continue
            
        latt_mx[i,:] = latt_mx[i,:] * new_times_oldcell[i]
    
    lattice = Lattice(latt_mx)
    # Generate structure
    slab = Structure(lattice, species_tot, coords_tot, coords_are_cartesian=True)
    
    return slab

### Form I Paracetamol Surface (010)
The same_layer_space is set to b of cif file. 

In [2]:
# Miller index of the slab
index = [0, 0, 1]
# Lattice parameter of the non-periodic dimension
same_layer_space = 12.667
# Stacking layers
layer = 4
# Independent layer files
layer_files = ['010-1.vasp', '010-2.vasp']
# Thickness of the vacuum layer in angstrom
vacuum = 25
# Output CIF file name
outputname = 'slab5-010.cif'

from pymatgen.io.cif import CifWriter


slab = get_slab(layer_files, index, layer, same_layer_space, vacuum)
CifWriter(slab).write_file(outputname)

### Form I Paracetamol Surface (011)

Although the code (should) support cutting surfaces that are not parallel to the cell surfaces, such surfaces bring difficulties to analysis. The unite cell is rotated by a matrix \[\[1, 0, 0\], \[0, 1, -1\], \[0, 1, 1\]\], making the \[011\] direction parallel to the new *b* direction and the (011) surface become the (010) in VESTA. 

In the following case, the index becomes (010) and the same_layer_size equals to the new *b*. In the following code,  *b* is divided by 2 because the lattice parameter along this direction is doubled to get a complete coordinates of both layers (i.e., atoms of the same layer should locate at the same side of the periodic boundary)

In [3]:
index = [0, 1, 0]
same_layer_space = ((10.91056)**2 + (29.30586)**2)**0.5 / 2
layer = 4
layer_files = ['011-1.vasp', '011-2.vasp']
vacuum = 25
outputname = 'slab4-011.cif'

from pymatgen.io.cif import CifWriter


slab = get_slab(layer_files, index, layer, same_layer_space, vacuum)
CifWriter(slab).write_file(outputname)